In [29]:
from urllib.request import Request, urlopen
import urllib
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import numpy as np

from datetime import datetime,date,timedelta  

import time

In [53]:
start_time = time.time()

day_stop = pd.to_datetime('01-01-2021' , format="%d-%m-%Y").date()
day = pd.to_datetime('01-01-2016' , format="%d-%m-%Y").date()


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
liste_titre = []
liste_date = []
liste_date_err_500 = [] # beaucoup de pages inaccessibles er500

e = False
numero_page = 1

while (day!= day_stop):
    try:
        day_str = day.strftime('%Y/%m/%d')
        url = "http://q.liberation.fr/archives/"+day_str+"/?page="+str(numero_page)


        req = Request(url , headers=headers)
        page = soup( urlopen(req).read() , "lxml")



        if page.find('div',{ 'class':"block-call-items"}).findAll('p') :
            #print('if: page numéro', numero_page)
            
            for titre in page.find('div',{ 'class':"block-call-items"}).findAll('p'):
                liste_titre.append(titre.find('a').getText().encode('utf-8').decode('utf-8').replace(u'\xa0', u' '))
                liste_date.append(day)
            numero_page +=1
            
        else:

            print(day," Temps Exec°: " ,(time.time() - start_time))
            day = day + timedelta(days=1)
            numero_page=1
    
    except urllib.error.HTTPError as exc:
        if str(exc) == "HTTP Error 500: Internal Server Error":
            liste_date_err_500.append(day)
            print(day , "Erreur500")
        day = day + timedelta(days=1)
        numero_page=1

2016-01-01  Temps Exec°:  0.9205760955810547
2016-01-02  Temps Exec°:  1.9881548881530762
2016-01-03  Temps Exec°:  4.178940534591675
2016-01-04  Temps Exec°:  5.1009202003479
2016-01-05  Temps Exec°:  6.600328207015991
2016-01-06  Temps Exec°:  7.832416296005249
2016-01-07  Temps Exec°:  8.812884330749512
2016-01-08  Temps Exec°:  10.070921897888184
2016-01-09  Temps Exec°:  10.533921957015991
2016-01-10  Temps Exec°:  11.252920866012573
2016-01-11  Temps Exec°:  12.489810705184937
2016-01-12  Temps Exec°:  13.993268013000488
2016-01-13  Temps Exec°:  14.930385112762451
2016-01-14  Temps Exec°:  16.107865571975708
2016-01-15  Temps Exec°:  17.444864988327026
2016-01-16  Temps Exec°:  17.855957508087158
2016-01-17  Temps Exec°:  18.594869375228882
2016-01-18  Temps Exec°:  19.80386996269226
2016-01-19  Temps Exec°:  20.857892751693726
2016-01-20  Temps Exec°:  21.592873334884644
2016-01-21  Temps Exec°:  22.604872465133667
2016-01-22  Temps Exec°:  23.92487359046936
2016-01-23  Temps E

In [54]:
liste_date_err_500

[datetime.date(2019, 9, 26),
 datetime.date(2019, 10, 1),
 datetime.date(2019, 10, 3),
 datetime.date(2019, 10, 10)]

In [57]:
df_lib_err = pd.DataFrame()
df_lib_err['date_err500'] = liste_date_err_500
df_lib_err.to_csv(index=False ,path_or_buf='liberation_date_err500.csv', encoding = 'utf-8')

In [56]:
df_lib = pd.DataFrame()
#liste_titre = []
#liste_titre.append(pd.to_datetime(day , format="%d-%m-%Y").date())
df_lib['titre'] = liste_titre
df_lib['date'] = liste_date
df_lib.to_csv(index=False ,path_or_buf='data_scrapping_csv/liberation.csv', encoding = 'utf-8')